In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml.html as lh
from PIL import Image
import urllib.request
import urllib
import pickle

In [61]:
with open('benigns_df.pkl', 'rb') as handle:
    benigns_df = pickle.load(handle)
    
with open('benigns_abnormalities_df.pkl', 'rb') as handle:
    benigns_abnormalities_df = pickle.load(handle)
    
with open('cancers_df.pkl', 'rb') as handle:
    cancers_df = pickle.load(handle)
    
with open('cancers_abnormalities_df.pkl', 'rb') as handle:
    cancers_abnormalities_df = pickle.load(handle)
    
with open('normals_df.pkl', 'rb') as handle:
    normals_df = pickle.load(handle)    

In [110]:
# GUI - wyświetlanie 4 obrazków + wybieranie tego co jest akurat interesujących

## Normal

In [62]:
normals_df.columns

Index(['</pre>]', 'DATE_DIGITIZED', 'DATE_OF_STUDY', 'DENSITY', 'DIGITIZER',
       'FILM', 'FILM_TYPE', 'LEFT_CC', 'LEFT_MLO', 'PATIENT_AGE', 'RIGHT_CC',
       'RIGHT_MLO', '[<pre>', 'filename', 'ics_version', 'SEQUENCE'],
      dtype='object')

In [63]:
normals_df = normals_df.drop(['</pre>]', '[<pre>','FILM','SEQUENCE'], axis=1)

In [65]:
normals_df.head(3)

,DATE_DIGITIZED,DATE_OF_STUDY,DENSITY,DIGITIZER,FILM_TYPE,LEFT_CC,LEFT_MLO,PATIENT_AGE,RIGHT_CC,RIGHT_MLO,filename,ics_version
0,2 11 1995,5 3 1991,2,DBA 21,REGULAR,LINES 4349 PIXELS_PER_LINE 1979 BITS_PER_PIXEL...,LINES 5069 PIXELS_PER_LINE 2039 BITS_PER_PIXEL...,63,LINES 4334 PIXELS_PER_LINE 1874 BITS_PER_PIXEL...,LINES 5084 PIXELS_PER_LINE 2129 BITS_PER_PIXEL...,A-0002-1,1.0
1,10 8 1995,7 3 1991,3,DBA 21,REGULAR,LINES 4514 PIXELS_PER_LINE 2084 BITS_PER_PIXEL...,LINES 5159 PIXELS_PER_LINE 2069 BITS_PER_PIXEL...,66,LINES 4679 PIXELS_PER_LINE 2114 BITS_PER_PIXEL...,LINES 5399 PIXELS_PER_LINE 2159 BITS_PER_PIXEL...,A-0003-1,1.0
2,15 8 1995,23 10 1991,4,DBA 21,REGULAR,LINES 3959 PIXELS_PER_LINE 1799 BITS_PER_PIXEL...,LINES 4454 PIXELS_PER_LINE 1949 BITS_PER_PIXEL...,44,LINES 4139 PIXELS_PER_LINE 1904 BITS_PER_PIXEL...,LINES 5099 PIXELS_PER_LINE 2084 BITS_PER_PIXEL...,A-0005-1,1.0


In [73]:
print("Wszystkie prawdidłowe przypadki: " + str(len(normals_df)))
print("Prawdidłowe przypadki o gęstości 4: " + str(len(normals_df.loc[normals_df['DENSITY']=="4"])))

Wszystkie prawdidłowe przypadki: 694
Prawdidłowe przypadki o gęstości 4: 150


## Cancer & benign

In [75]:
benigns_df.columns

Index(['</pre>]', 'DATE_DIGITIZED', 'DATE_OF_STUDY', 'DENSITY', 'DIGITIZER',
       'FILM', 'FILM_TYPE', 'LEFT_CC', 'LEFT_MLO', 'PATIENT_AGE', 'RIGHT_CC',
       'RIGHT_MLO', 'SEQUENCE', '[<pre>', 'filename', 'ics_version'],
      dtype='object')

In [76]:
benigns_df = benigns_df.drop(['</pre>]','[<pre>', 'FILM', 'SEQUENCE'], axis=1)
cancers_df = cancers_df.drop(['</pre>]','[<pre>', 'FILM', 'SEQUENCE'], axis=1)

In [77]:
benigns_df.head(3)

,DATE_DIGITIZED,DATE_OF_STUDY,DENSITY,DIGITIZER,FILM_TYPE,LEFT_CC,LEFT_MLO,PATIENT_AGE,RIGHT_CC,RIGHT_MLO,filename,ics_version
0,9 10 1997,2 3 1993,3,LUMISYS LASER,REGULAR,LINES 4648 PIXELS_PER_LINE 2672 BITS_PER_PIXEL...,LINES 4680 PIXELS_PER_LINE 2664 BITS_PER_PIXEL...,66,LINES 4704 PIXELS_PER_LINE 2816 BITS_PER_PIXEL...,LINES 4648 PIXELS_PER_LINE 2864 BITS_PER_PIXEL...,C-0029-1,1.0
1,10 10 1997,18 11 1993,3,LUMISYS LASER,REGULAR,LINES 4672 PIXELS_PER_LINE 2616 BITS_PER_PIXEL...,LINES 4664 PIXELS_PER_LINE 2424 BITS_PER_PIXEL...,60,LINES 4656 PIXELS_PER_LINE 2680 BITS_PER_PIXEL...,LINES 4680 PIXELS_PER_LINE 2752 BITS_PER_PIXEL...,C-0033-1,1.0
2,14 4 1998,23 2 1996,2,LUMISYS LASER,REGULAR,LINES 6000 PIXELS_PER_LINE 3264 BITS_PER_PIXEL...,LINES 5992 PIXELS_PER_LINE 3368 BITS_PER_PIXEL...,56,LINES 5960 PIXELS_PER_LINE 3536 BITS_PER_PIXEL...,LINES 5944 PIXELS_PER_LINE 3752 BITS_PER_PIXEL...,C-0217-1,1.0


In [78]:
benigns_abnormalities_df.columns

Index(['</font></pre>,', '</font></pre>]', '<font', 'ASSESSMENT', 'BOUNDARY',
       'FILE:', 'LESION_TYPE', 'PATHOLOGY', 'SUBTLETY', 'TOTAL_ABNORMALITIES',
       'TOTAL_OUTLINES', '[<pre>', '</font><font', 'CORE', 'CLUSTERED'],
      dtype='object')

In [79]:
cancers_abnormalities_df.columns

Index(['</font></pre>,', '</font></pre>]', '<font', 'ASSESSMENT', 'BOUNDARY',
       'FILE:', 'LESION_TYPE', 'PATHOLOGY', 'SUBTLETY', 'TOTAL_ABNORMALITIES',
       'TOTAL_OUTLINES', '[<pre>', '</font><font', 'CORE'],
      dtype='object')

In [80]:
benigns_abnormalities_df = benigns_abnormalities_df.drop(['</font></pre>,', '</font></pre>]', '<font', 'BOUNDARY',\
                                                          '[<pre>', '</font><font', 'CORE', 'CLUSTERED'], axis=1)
cancers_abnormalities_df = cancers_abnormalities_df.drop(['</font></pre>,', '</font></pre>]', '<font', 'BOUNDARY',\
                                                          '[<pre>', '</font><font', 'CORE'], axis=1)

In [81]:
benigns_abnormalities_df.head(3)

,ASSESSMENT,FILE:,LESION_TYPE,PATHOLOGY,SUBTLETY,TOTAL_ABNORMALITIES,TOTAL_OUTLINES
0,3,C_0029_1.LEFT_CC.OVERLAY,MASS SHAPE OVAL MARGINS ILL_DEFINED,BENIGN,3,1,1
1,4,C_0033_1.RIGHT_CC.OVERLAY,MASS SHAPE OVAL MARGINS MICROLOBULATED,BENIGN,4,1,1
2,3,C_0217_1.RIGHT_CC.OVERLAY,MASS SHAPE ROUND MARGINS CIRCUMSCRIBED,BENIGN,5,1,1


In [104]:
cancers_abnormalities_df['LESION_TYPE'].unique()

array(['MASS SHAPE IRREGULAR MARGINS SPICULATED',
       'CALCIFICATION TYPE PLEOMORPHIC DISTRIBUTION SEGMENTAL',
       'CALCIFICATION TYPE AMORPHOUS DISTRIBUTION CLUSTERED',
       'CALCIFICATION TYPE PLEOMORPHIC DISTRIBUTION CLUSTERED',
       'MASS SHAPE LOBULATED MARGINS CIRCUMSCRIBED',
       'MASS SHAPE IRREGULAR MARGINS ILL_DEFINED',
       'MASS SHAPE LOBULATED MARGINS ILL_DEFINED',
       'MASS SHAPE ARCHITECTURAL_DISTORTION MARGINS SPICULATED',
       'MASS SHAPE IRREGULAR-ARCHITECTURAL_DISTORTION MARGINS SPICULATED',
       'CALCIFICATION TYPE FINE_LINEAR_BRANCHING DISTRIBUTION LINEAR-SEGMENTAL',
       'CALCIFICATION TYPE PUNCTATE-AMORPHOUS-PLEOMORPHIC DISTRIBUTION CLUSTERED',
       'CALCIFICATION TYPE PLEOMORPHIC DISTRIBUTION REGIONAL',
       'CALCIFICATION TYPE AMORPHOUS-PLEOMORPHIC DISTRIBUTION CLUSTERED',
       'MASS SHAPE LOBULATED-ARCHITECTURAL_DISTORTION MARGINS MICROLOBULATED-SPICULATED',
       'MASS SHAPE LOBULATED MARGINS CIRCUMSCRIBED-ILL_DEFINED',
       'M

In [109]:
# tylko mass, odrzucić shape: oval i round

In [90]:
benigns_abnormalities_df['filename'] = benigns_abnormalities_df.apply(lambda row: row['FILE:'].split('.')[0].replace('_','-'), axis=1)
cancers_abnormalities_df['filename'] = cancers_abnormalities_df.apply(lambda row: row['FILE:'].split('.')[0].replace('_','-'), axis=1)

In [95]:
len(benigns_df)

855

In [97]:
len(benigns_abnormalities_df)

855

In [98]:
len(cancers_df)

914

In [99]:
len(cancers_abnormalities_df)

914

In [103]:
len(benigns_abnormalities_df[benigns_abnormalities_df['LESION_TYPE'].str.contains("MARGINS SPICULATED")])

12

In [106]:
len(benigns_abnormalities_df[benigns_abnormalities_df['LESION_TYPE'].str.contains("ARCHITECTURAL_DISTORTION")])

13

In [107]:
len(cancers_abnormalities_df[cancers_abnormalities_df['LESION_TYPE'].str.contains("MARGINS SPICULATED")])

227

In [108]:
len(cancers_abnormalities_df[cancers_abnormalities_df['LESION_TYPE'].str.contains("ARCHITECTURAL_DISTORTION")])

90